In [1]:
import numpy as np
from flask import Flask, request

# Numpy Feed Forward NN

In [2]:
# Define the Level class as structure for the NN (layers)
class Level:
    def __init__(self, input_count, output_count):
        self.inputs = np.zeros(input_count)
        self.outputs = np.zeros(output_count)
        self.biases = np.random.uniform(-1, 1, output_count)
        self.weights = np.random.uniform(-1, 1, (input_count, output_count))

    def feed_forward(self, given_inputs):
        self.inputs = given_inputs
        sum = np.dot(self.inputs, self.weights)
        self.outputs = np.where(sum > self.biases, 1, 0)
        return self.outputs

# Define the NN
class NeuralNetwork:
    def __init__(self, neuron_counts):
        self.levels = [Level(neuron_counts[i], neuron_counts[i+1]) for i in range(len(neuron_counts)-1)]

    def feed_forward(self, given_inputs):
        outputs = self.levels[0].feed_forward(given_inputs)
        for i in range(1, len(self.levels)):
            outputs = self.levels[i].feed_forward(outputs)
        return outputs

    def mutate(self, amount=1):
        for level in self.levels:
            level.biases = self.lerp(level.biases, np.random.uniform(-1, 1, len(level.biases)), amount)
            for i in range(len(level.weights)):
                for j in range(len(level.weights[i])):
                    level.weights[i][j] = self.lerp(level.weights[i][j], np.random.uniform(-1, 1), amount)

    def lerp(self, start, end, amount):
        return (1 - amount) * start + amount * end

# API Endpoint (Frontend -> Backend -> Frontend)

 The /drive endpoint accepts POST requests with sensor data from the initialized car (given_inputs). The given_inputs should be a list or a 1-dimensional numpy array of numbers, where each number represents the distance from the car to an obstacle detected by a sensor ray. The length of this list or array should match the number of input neurons in the neural network (in this case 5 input nodes).

 The input_array comes from the sensor data by the Sensor class which uses ray casting to detect obstacles around the car. For each ray (5), it calculates the intersection with the road borders and traffic and returns the distance to the closest intersection point. This distance is then used as an input for the neural network.

- given_inputs[0] = Distance to the closest obstacle detected by the first ray (leftmost)
- given_inputs[1] = Distance to the closest obstacle detected by the second ray
- given_inputs[2] = Distance to the closest obstacle detected by the third ray
- given_inputs[3] = Distance to the closest obstacle detected by the fourth ray
- given_inputs[4] = Distance to the closest obstacle detected by the fifth ray (rightmost)

 It feeds the sensor data through the neural network and returns the actions (outputs) as a response. The outputs is a list of binary values (0 or 1), where each value corresponds to a control action for the car:

 - outputs[0] = controls.forward, 
 - outputs[1] = controls.left, 
 - outputs[2] = controls.right, 
 - outputs[3] = controls.reverse. 


In the video he uses 5 input nodes, 6 nodes in one single hidden layer and 4 output nodes into the car class.

The outputs of the NN are fed to the update method. Then we must connect the outputs to the controls (control.js) in the main.js ("AI"). The car class gets an attribute useBrain with controltype = "AI".

Initialize 100 cars and store the best performing car brain (bestCar, bestBrain) in localStorage (highest up on straight roads as fitness function (RL!)). Mutate the best performing car by x%.
 

## PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 6)  # 5 input nodes, 6 nodes in hidden layer
        self.fc2 = nn.Linear(6, 4)  # 6 nodes in hidden layer, 4 output nodes

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Call NN 

Must be integrated into main.

Send input to flask server and NN:

In [12]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/drive', methods=['POST'])
def drive():
    data = request.get_json()
    given_inputs = torch.tensor(data['given_inputs']).float()
    net = Net()
    outputs = net(given_inputs)
    return jsonify(outputs.tolist())

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.127:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Apr/2024 18:28:38] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Apr/2024 18:28:38] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.127 - - [24/Apr/2024 18:28:42] "GET / HTTP/1.1" 404 -
192.168.0.127 - - [24/Apr/2024 18:28:42] "GET /favicon.ico HTTP/1.1" 404 -


Send output to frontend

In [ ]:
let given_inputs = [/* sensor data */];
fetch('/drive', {
  method: 'POST',
  headers: {
    'Content-Type': 'application/json',
  },
  body: JSON.stringify({given_inputs}),
})
.then(response => response.json())
.then(outputs => {
  // Use the outputs to control the car
});